# MP（限界利益）予測モデルの比較実験

複数の機械学習モデルを試して、最も精度の高いモデルを特定します。

## 比較するモデル
1. **線形回帰** (Linear Regression)
2. **Ridge回帰** (L2正則化)
3. **Lasso回帰** (L1正則化)
4. **ランダムフォレスト** (Random Forest)
5. **勾配ブースティング** (Gradient Boosting)
6. **XGBoost**
7. **LightGBM**
8. **Prophet**（時系列特化）

## 1. ライブラリのインストールとインポート

In [ ]:
# 必要なライブラリのインストール
!pip install xgboost lightgbm prophet -q

# 日本語フォント（Colab用）
!apt-get -y install fonts-ipafont-gothic > /dev/null 2>&1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', family='IPAGothic')

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet

import warnings
warnings.filterwarnings('ignore')

print("ライブラリの準備完了！")

## 2. データの読み込みと前処理

In [ ]:
# ファイルのアップロード
from google.colab import files
uploaded = files.upload()

In [ ]:
# データの読み込み
if len(uploaded) == 0:
    raise ValueError("エラー: ファイルがアップロードされていません。上のセルを実行してファイルをアップロードしてください。")

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, encoding='utf-8')

print(f"データ形状: {df.shape}")
print(f"\n列名一覧:")
print(df.columns.tolist())

In [ ]:
# 列名の空白を削除（念のため）
df.columns = df.columns.str.strip()

# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])

# MPの前処理（カンマと引用符を除去して数値に変換）
df['MP'] = df['MP'].astype(str).str.replace(',', '').str.replace('"', '').str.strip()
df['MP'] = pd.to_numeric(df['MP'], errors='coerce')

print("データの先頭5行:")
df.head()

In [ ]:
# MPが存在するデータ（訓練用）と存在しないデータ（予測用）を分離
df_train_full = df[df['MP'].notna()].copy()
df_predict = df[df['MP'].isna()].copy()

print(f"訓練用データ: {len(df_train_full)}行 ({df_train_full['date'].min()} 〜 {df_train_full['date'].max()})")
print(f"予測用データ: {len(df_predict)}行 ({df_predict['date'].min()} 〜 {df_predict['date'].max()})")

# MP の基本統計
print(f"\nMP の基本統計:")
print(f"  平均: {df_train_full['MP'].mean():,.0f}")
print(f"  標準偏差: {df_train_full['MP'].std():,.0f}")
print(f"  最小: {df_train_full['MP'].min():,.0f}")
print(f"  最大: {df_train_full['MP'].max():,.0f}")

## 3. 特徴量の準備

In [ ]:
# 特徴量として使用する列を選択（date, day, MP以外のダミー変数列）
feature_cols = [col for col in df.columns if col not in ['date', 'day', 'MP']]

print(f"使用する特徴量 ({len(feature_cols)}個):")
print(feature_cols)

In [ ]:
# 追加の特徴量を作成（時系列的な特徴）
def add_time_features(df):
    df = df.copy()
    df['day_of_year'] = df['date'].dt.dayofyear
    df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['date'].dt.quarter
    
    # 周期的な特徴（sin/cos変換）
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    df['week_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)
    
    return df

df_train_full = add_time_features(df_train_full)
df_predict = add_time_features(df_predict)

# 更新された特徴量リスト
feature_cols_extended = feature_cols + ['day_of_year', 'week_of_year', 'quarter', 
                                         'day_sin', 'day_cos', 'week_sin', 'week_cos']

print(f"\n拡張特徴量 ({len(feature_cols_extended)}個):")
print(feature_cols_extended)

## 4. 訓練データとテストデータの分割

時系列データなので、最後の一定期間をテストデータとして使用

In [ ]:
# テスト期間の設定（最後の6ヶ月＝約180日）
test_days = 180

# 時系列順にソート
df_train_full = df_train_full.sort_values('date').reset_index(drop=True)

# 分割
train_df = df_train_full.iloc[:-test_days].copy()
test_df = df_train_full.iloc[-test_days:].copy()

print(f"訓練データ: {len(train_df)}日 ({train_df['date'].min().date()} 〜 {train_df['date'].max().date()})")
print(f"テストデータ: {len(test_df)}日 ({test_df['date'].min().date()} 〜 {test_df['date'].max().date()})")

# 特徴量と目的変数の分離
X_train = train_df[feature_cols_extended]
y_train = train_df['MP']
X_test = test_df[feature_cols_extended]
y_test = test_df['MP']

print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

## 5. モデルの定義と学習

In [ ]:
# 評価指標を計算する関数
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    r2 = r2_score(y_true, y_pred)
    
    return {
        'モデル': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE(%)': mape,
        'R2': r2
    }

In [ ]:
# モデルの辞書
models = {
    '1. 線形回帰': LinearRegression(),
    '2. Ridge回帰': Ridge(alpha=1.0),
    '3. Lasso回帰': Lasso(alpha=1.0),
    '4. ランダムフォレスト': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    '5. 勾配ブースティング': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    '6. XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42),
    '7. LightGBM': lgb.LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, verbose=-1)
}

# 結果を格納するリスト
results = []
predictions = {}

print("モデルの学習と評価を開始...\n")
print("="*70)

for name, model in models.items():
    print(f"\n{name} を学習中...")
    
    # 学習
    model.fit(X_train, y_train)
    
    # 予測
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    
    # 評価
    result = evaluate_model(y_test, y_pred, name)
    results.append(result)
    
    print(f"  MAPE: {result['MAPE(%)']:.2f}%, R2: {result['R2']:.4f}")

print("\n" + "="*70)
print("全モデルの学習完了！")

## 6. Prophet モデル（時系列特化）

In [ ]:
print("\n8. Prophet を学習中...")

# Prophet用のデータ準備
prophet_train = train_df[['date', 'MP']].rename(columns={'date': 'ds', 'MP': 'y'})
prophet_test = test_df[['date', 'MP']].rename(columns={'date': 'ds', 'MP': 'y'})

# 祝日フラグを追加変数として使用
prophet_train['holiday_flg'] = train_df['holiday flg'].values

# Prophetモデルの作成
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='multiplicative'
)

# 追加のリグレッサーとして祝日フラグを追加
prophet_model.add_regressor('holiday_flg')

# 学習
prophet_model.fit(prophet_train)

# 予測用データフレームの作成
future = prophet_test[['ds']].copy()
future['holiday_flg'] = test_df['holiday flg'].values

# 予測
prophet_forecast = prophet_model.predict(future)
y_pred_prophet = prophet_forecast['yhat'].values
predictions['8. Prophet'] = y_pred_prophet

# 評価
result_prophet = evaluate_model(y_test, y_pred_prophet, '8. Prophet')
results.append(result_prophet)

print(f"  MAPE: {result_prophet['MAPE(%)']:.2f}%, R2: {result_prophet['R2']:.4f}")

## 7. 結果の比較

In [ ]:
# 結果をDataFrameに変換
results_df = pd.DataFrame(results)

# MAPEでソート（小さい順）
results_df = results_df.sort_values('MAPE(%)')

# 表示用にフォーマット
results_display = results_df.copy()
results_display['MAE'] = results_display['MAE'].apply(lambda x: f"{x:,.0f}")
results_display['RMSE'] = results_display['RMSE'].apply(lambda x: f"{x:,.0f}")
results_display['MAPE(%)'] = results_display['MAPE(%)'].apply(lambda x: f"{x:.2f}")
results_display['R2'] = results_display['R2'].apply(lambda x: f"{x:.4f}")

print("\n" + "="*80)
print("モデル比較結果（MAPE順）")
print("="*80)
print("\n※ MAPE（平均絶対パーセント誤差）が小さいほど精度が高い\n")
print(results_display.to_string(index=False))

In [ ]:
# 最良モデルの特定
best_model_name = results_df.iloc[0]['モデル']
best_mape = results_df.iloc[0]['MAPE(%)']

print("\n" + "*"*80)
print(f"🏆 最も精度が高いモデル: {best_model_name}")
print(f"   MAPE: {best_mape:.2f}%")
print("*"*80)

In [ ]:
# 精度比較のグラフ
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MAPE比較
ax1 = axes[0]
colors = ['gold' if m == best_model_name else 'steelblue' for m in results_df['モデル']]
bars = ax1.barh(results_df['モデル'], results_df['MAPE(%)'], color=colors)
ax1.set_xlabel('MAPE (%)', fontsize=12)
ax1.set_title('モデル別 MAPE（小さいほど良い）', fontsize=14)
ax1.invert_yaxis()
for bar, val in zip(bars, results_df['MAPE(%)']):
    ax1.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.2f}%', va='center')

# R2比較
ax2 = axes[1]
colors = ['gold' if m == best_model_name else 'steelblue' for m in results_df['モデル']]
bars = ax2.barh(results_df['モデル'], results_df['R2'], color=colors)
ax2.set_xlabel('R² スコア', fontsize=12)
ax2.set_title('モデル別 R²（大きいほど良い）', fontsize=14)
ax2.invert_yaxis()
for bar, val in zip(bars, results_df['R2']):
    ax2.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', va='center')

plt.tight_layout()
plt.show()

## 8. 予測結果の可視化

In [ ]:
# 上位3モデルの予測を可視化
top_3_models = results_df.head(3)['モデル'].tolist()

fig, ax = plt.subplots(figsize=(14, 6))

# 実績
ax.plot(test_df['date'], y_test / 1_000_000, 'k-', label='実績', linewidth=2, alpha=0.8)

# 上位3モデルの予測
colors = ['red', 'blue', 'green']
for i, model_name in enumerate(top_3_models):
    ax.plot(test_df['date'], predictions[model_name] / 1_000_000, 
            '--', color=colors[i], label=model_name, linewidth=1.5, alpha=0.7)

ax.set_title('テスト期間における予測と実績の比較（上位3モデル）', fontsize=14)
ax.set_xlabel('日付')
ax.set_ylabel('MP（百万円）')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 最良モデルの予測を詳細表示
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

best_pred = predictions[best_model_name]

# 上: 予測 vs 実績
ax1 = axes[0]
ax1.plot(test_df['date'], y_test / 1_000_000, 'b-', label='実績', linewidth=2)
ax1.plot(test_df['date'], best_pred / 1_000_000, 'r--', label=f'{best_model_name}予測', linewidth=2)
ax1.fill_between(test_df['date'], y_test / 1_000_000, best_pred / 1_000_000, alpha=0.3, color='gray')
ax1.set_title(f'最良モデル（{best_model_name}）の予測 vs 実績', fontsize=14)
ax1.set_ylabel('MP（百万円）')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 下: 予測誤差
ax2 = axes[1]
error_pct = (best_pred - y_test) / y_test * 100
colors = ['red' if e > 0 else 'blue' for e in error_pct]
ax2.bar(test_df['date'], error_pct, color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.axhline(y=error_pct.mean(), color='green', linestyle='--', label=f'平均誤差: {error_pct.mean():.2f}%')
ax2.set_title('日別予測誤差（%）', fontsize=14)
ax2.set_ylabel('誤差（%）')
ax2.set_xlabel('日付')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 9. 特徴量の重要度（ツリー系モデル）

In [ ]:
# LightGBMの特徴量重要度を表示
lgb_model = models['7. LightGBM']

# 特徴量重要度を取得
importance = pd.DataFrame({
    '特徴量': feature_cols_extended,
    '重要度': lgb_model.feature_importances_
}).sort_values('重要度', ascending=False)

# 上位15個を表示
fig, ax = plt.subplots(figsize=(10, 8))
top_n = 15
ax.barh(importance.head(top_n)['特徴量'], importance.head(top_n)['重要度'], color='steelblue')
ax.set_xlabel('重要度')
ax.set_title(f'特徴量重要度（LightGBM, 上位{top_n}個）', fontsize=14)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\n特徴量重要度（上位15個）:")
print(importance.head(15).to_string(index=False))

## 10. 将来予測（2025-2026年）の実行

In [ ]:
# 全訓練データで最良モデルを再学習
print(f"最良モデル（{best_model_name}）を全データで再学習...")

X_full = df_train_full[feature_cols_extended]
y_full = df_train_full['MP']

# モデルの再作成と学習
if 'LightGBM' in best_model_name:
    final_model = lgb.LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, verbose=-1)
elif 'XGBoost' in best_model_name:
    final_model = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
elif 'ランダムフォレスト' in best_model_name:
    final_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
elif '勾配ブースティング' in best_model_name:
    final_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
elif 'Ridge' in best_model_name:
    final_model = Ridge(alpha=1.0)
elif 'Lasso' in best_model_name:
    final_model = Lasso(alpha=1.0)
else:
    final_model = LinearRegression()

final_model.fit(X_full, y_full)
print("再学習完了！")

In [ ]:
# 将来データの予測
if len(df_predict) > 0:
    X_future = df_predict[feature_cols_extended]
    future_predictions = final_model.predict(X_future)
    
    # 結果をDataFrameに追加
    df_predict['MP_予測'] = future_predictions
    
    print("\n将来予測結果（月別集計）:")
    df_predict['年月'] = df_predict['date'].dt.to_period('M')
    monthly_forecast = df_predict.groupby('年月')['MP_予測'].sum().reset_index()
    monthly_forecast['MP_予測（百万円）'] = (monthly_forecast['MP_予測'] / 1_000_000).round(1)
    print(monthly_forecast[['年月', 'MP_予測（百万円）']].to_string(index=False))
    
    print(f"\n予測期間合計: {df_predict['MP_予測'].sum() / 1_000_000:,.1f} 百万円")
else:
    print("予測対象データがありません")

In [ ]:
# 将来予測の可視化
if len(df_predict) > 0:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # 過去の実績
    ax.plot(df_train_full['date'], df_train_full['MP'] / 1_000_000, 'b-', 
            label='実績', linewidth=1, alpha=0.7)
    
    # 将来予測
    ax.plot(df_predict['date'], df_predict['MP_予測'] / 1_000_000, 'r--', 
            label='予測', linewidth=1.5)
    
    # 境界線
    ax.axvline(x=df_train_full['date'].max(), color='gray', linestyle=':', alpha=0.7)
    
    ax.set_title('MP予測結果（実績 + 将来予測）', fontsize=14)
    ax.set_xlabel('日付')
    ax.set_ylabel('MP（百万円）')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 11. 予測結果のエクスポート

In [ ]:
# 予測結果をCSVにエクスポート
if len(df_predict) > 0:
    export_df = df_predict[['date', 'day', 'MP_予測']].copy()
    export_df['MP_予測'] = export_df['MP_予測'].round(0).astype(int)
    export_df.to_csv('mp_forecast_result.csv', index=False, encoding='utf-8-sig')
    print("予測結果を 'mp_forecast_result.csv' に保存しました")
    
    # ダウンロード
    files.download('mp_forecast_result.csv')

## 12. まとめ

In [ ]:
print("\n" + "="*80)
print("分析まとめ")
print("="*80)

print("\n【データ概要】")
print(f"  訓練データ期間: {df_train_full['date'].min().date()} 〜 {df_train_full['date'].max().date()}")
print(f"  予測対象期間: {df_predict['date'].min().date()} 〜 {df_predict['date'].max().date()}")

print("\n【モデル比較結果（MAPE順）】")
print(results_display.to_string(index=False))

print(f"\n【最良モデル】")
print(f"  {best_model_name}")
print(f"  MAPE: {best_mape:.2f}%")

print("\n【考察】")
print("""
- ダミー変数（年・月・曜日・祝日）を使った特徴量は日次予測に有効
- 周期的な特徴（sin/cos変換）を追加することで季節性を捉えやすくなる
- ツリー系モデル（LightGBM, XGBoost等）は特徴量の非線形関係を捉えやすい
- Prophetは祝日効果や季節性を自動で検出するが、詳細な特徴量がある場合は
  他のモデルの方が精度が高くなることもある
""")